In [84]:
import pandas as pd
from datetime import datetime

In [85]:
df = pd.read_csv('../db/complete_vader.txt',sep='|')
df.head()

,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment
0,0,0,NaN,"Good evening, this is Michael Partridge, Seni...",VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.4404
1,1,1,NaN,"Tonight, we will review with you Vertex's busi...",VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.4215
2,2,2,NaN,"Making prepared remarks on the call tonight, w...",VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.2263
3,3,3,NaN,We recommend that you access the webcast slide...,VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.3612
4,4,4,NaN,This conference call is being recorded and a r...,VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.0000


In [86]:
# add new datetime column by parsing the date and time out of the call title

df['call_datetime'] = df['call_title'].apply(lambda x: datetime.strptime(x.split('Conference Call')[1].lstrip()[0:-3],
                                        '%B %d, %Y  %I:%M %p'))

df['call_date'] = pd.to_datetime(df['call_datetime']).dt.date
df['call_time'] = pd.to_datetime(df['call_datetime']).dt.time

df.head()



,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment,call_datetime,call_date,call_time
0,0,0,NaN,"Good evening, this is Michael Partridge, Seni...",VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.4404,2019-10-30 17:00:00,2019-10-30,17:00:00
1,1,1,NaN,"Tonight, we will review with you Vertex's busi...",VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.4215,2019-10-30 17:00:00,2019-10-30,17:00:00
2,2,2,NaN,"Making prepared remarks on the call tonight, w...",VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.2263,2019-10-30 17:00:00,2019-10-30,17:00:00
3,3,3,NaN,We recommend that you access the webcast slide...,VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.3612,2019-10-30 17:00:00,2019-10-30,17:00:00
4,4,4,NaN,This conference call is being recorded and a r...,VRTX,Vertex Pharmaceuticals Incorporated (NASDAQ:VR...,Michael Partridge,presentation,0.0000,2019-10-30 17:00:00,2019-10-30,17:00:00


In [87]:
df = df.set_index('call_datetime')
df['unique_id'] = df['id'].astype(str)+df['ticker']
df = df.drop_duplicates(subset='unique_id',keep='last')
df.count()


Unnamed: 0         45876
id                 45876
sentiment              0
sentence           45876
ticker             45876
call_title         45876
speaker            45876
call_section       45876
vader_sentiment    45876
call_date          45876
call_time          45876
unique_id          45876
dtype: int64

In [88]:
morning_df = df.between_time('00:00','14:00')
morning_df['base_date'] = (morning_df['call_date'] + pd.DateOffset(-1))
morning_df['next_date'] = morning_df['call_date']

morning_df.count()


/Users/andrewbehrman/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/andrewbehrman/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Unnamed: 0         36556
id                 36556
sentiment              0
sentence           36556
ticker             36556
call_title         36556
speaker            36556
call_section       36556
vader_sentiment    36556
call_date          36556
call_time          36556
unique_id          36556
base_date          36556
next_date          36556
dtype: int64

In [89]:

morning_df['base_day_of_week'] = morning_df['base_date'].dt.dayofweek
trim_df = morning_df.loc[morning_df['base_day_of_week'] == 6]
trim_df['base_date'] = (trim_df['call_date'] + pd.DateOffset(-3))
trim_df = trim_df.drop(columns=['base_day_of_week'])
morning_df = morning_df.drop(columns=['base_day_of_week'])


trim_df.head()

/Users/andrewbehrman/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/andrewbehrman/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment,call_date,call_time,unique_id,base_date,next_date
call_datetime,,,,,,,,,,,,,,
2019-11-04 08:30:00,4041,4041,NaN,"Ladies and gentlemen, thank you for standing ...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,0.6705,2019-11-04,08:30:00,4041UAA,2019-11-01,2019-11-04
2019-11-04 08:30:00,4042,4042,NaN,"At this time, all participants are in a listen...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,0.0000,2019-11-04,08:30:00,4042UAA,2019-11-01,2019-11-04
2019-11-04 08:30:00,4043,4043,NaN,"After the speaker presentation, there will be ...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,0.0000,2019-11-04,08:30:00,4043UAA,2019-11-01,2019-11-04
2019-11-04 08:30:00,4044,4044,NaN,[Operator Instructions].,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,0.0000,2019-11-04,08:30:00,4044UAA,2019-11-01,2019-11-04
2019-11-04 08:30:00,4045,4045,NaN,Please be advised that today's conference is b...,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,0.3182,2019-11-04,08:30:00,4045UAA,2019-11-01,2019-11-04


In [90]:
morning_df = morning_df.append(trim_df)
morning_df = morning_df.drop_duplicates(subset='unique_id',keep='last')
morning_df.count()


Unnamed: 0         36556
id                 36556
sentiment              0
sentence           36556
ticker             36556
call_title         36556
speaker            36556
call_section       36556
vader_sentiment    36556
call_date          36556
call_time          36556
unique_id          36556
base_date          36556
next_date          36556
dtype: int64

In [92]:
afternoon_df = df.between_time('14:00','23:59')
afternoon_df['base_date'] = afternoon_df['call_date']
afternoon_df['next_date'] = (afternoon_df['call_date'] + pd.DateOffset(1))

afternoon_df.count()

/Users/andrewbehrman/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/andrewbehrman/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Unnamed: 0         9320
id                 9320
sentiment             0
sentence           9320
ticker             9320
call_title         9320
speaker            9320
call_section       9320
vader_sentiment    9320
call_date          9320
call_time          9320
unique_id          9320
base_date          9320
next_date          9320
dtype: int64

In [93]:
all_times_df = morning_df.append(afternoon_df)
all_times_df


,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,vader_sentiment,call_date,call_time,unique_id,base_date,next_date
call_datetime,,,,,,,,,,,,,,
2019-11-01 10:00:00,457,457,NaN,Good morning and welcome to the Dominion Ener...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.7906,2019-11-01,10:00:00,457D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,458,458,NaN,"At this time, each of your lines is in a liste...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,2019-11-01,10:00:00,458D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,459,459,NaN,"At the conclusion of today’s presentation, we ...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.0000,2019-11-01,10:00:00,459D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,460,460,NaN,[Operator Instructions] I would now like to tu...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Operator,operator_instruction,0.3612,2019-11-01,10:00:00,460D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,461,461,NaN,Good morning and welcome.,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.7096,2019-11-01,10:00:00,461D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,462,462,NaN,I encourage you to visit our Investor Relation...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.6369,2019-11-01,10:00:00,462D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,463,463,NaN,The Investor Relations team will be available ...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.0000,2019-11-01,10:00:00,463D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,464,464,NaN,Earnings materials including our prepared rema...,D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,-0.3818,2019-11-01,10:00:00,464D,2019-10-31 00:00:00,2019-11-01
2019-11-01 10:00:00,465,465,NaN,"Please refer to our SEC filings, including our...",D,"Dominion Energy, Inc. (NYSE:D) Q3 2019 Earning...",Steven Ridge,presentation,0.3182,2019-11-01,10:00:00,465D,2019-10-31 00:00:00,2019-11-01


In [94]:
# output result back to csv
all_times_df.to_csv('../db/sentences_sample_with_datetime2.csv', sep=',')